<a href="https://colab.research.google.com/github/pld000/z3/blob/main/z3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# III. Внутреннее устройство Z3. Доказательства

---

*Подготовлено Высшей школой программирования Сергея Бобровского 2023*

https://vk.com/lambda_brain

Ссылки на оригинальные ноутбуки в первом занятии.

In [2]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 60.6 MB/s eta 0:00:00


In [3]:
from z3 import *

# 1. Как Z3 работает

## 1.1. SAT/SMT-солверы

Очень полезно, полагают активные пользователи Z3, держать в голове хотя бы простую модель того, как этот инструмент работает, чтобы понимать, в каких ситуациях могут возникнуть трудности, а главное, учиться формулировать задачи так, чтобы способ их решения стал потенциально понятен солверу (и, по сути, вам самому). Это такая своеобразная форма прояснения процесса решения в интерактивном диалоге с решателем.

Z3 и все другие SMT-солверы базируются на ключевой теории -- **выполнимости логических формул** (SATisfiability problem, **SAT**).

**SMT** (**выполнимость формул в теориях**, satisfiability modulo theories) -- это обобщение SAT, когда вместо логических переменных в формулах используются предикаты из различных теорий (теории чисел различных типов, теории массивов, теории битовых векторов и т. д.). Поэтому подобные солверы нередко называются SAT/SMT-солверы.


## 1.2. SAT

SAT определяет выполнимость формул в пропозиционной логике.

**Пропозиционная логическая формула** -- это комбинация переменных, значений True/False и операций-связок And, Or, Implies и Not. "Подавая" значения переменным, мы получаем на выходе формулы true или false. Можно взять метафору логических схем.

## 1.3. SMT

Важный вопрос, который часто задаётся относительно пропозиционной формулы -- это её истинность. Истинность означает, что найдётся вариант значений переменных такой, при котором вся формула будет истинной (верной).

Поэтому SMT-солверы дедуктивно решают обратную задачу: не вычисление конечного значения формулы, а поиск таких входных значений, при которых она будет истинна. Трудоёмкость решения такой задачи в общем и целом относится к так называемому **классу сложности NP**, который подразумевает, что пространство поиска решения экспоненциально (грубым перебором его не атаковать), но каждый конкретный вариант решения проверяется быстро (формула вычисляется легко). В некотором смысле, требуется "угадать" верный вариант с помощью эвристик или различных математических методов. Что, впрочем, не исключает и brute force, если надо перебрать небольшое количество вариантов (как правило, когда переменных несколько десятков, и каждая, соответственно, может принимать только два значения).

SAT/SMT-солверы, конечно, блуждают по пространству решения достаточно разумно, и некоторые эвристики, кстати, используются в нейронных сетях (градиентный спуск). В частности, бум солверов начался немного раньше нейронных сетей, в конце 1990-х/начале 2000-х, когда были найдены качественно более продуктивные техники поиска решений: backjumping (сужение области поиска), two watched literal (очень эффективная техника, используемая сегодня практически во всех солверах, но крайне трудная для реализации -- за счёт использования связных списков указателей), conflict directed clause learning (поиск с обучением на графе выражения), и random restart (случайные прыжки по пространству решений, чтобы избегать локальных максимумов).

## 1.4. Устойство SMT-солвера

В SMT-солверах обычно выделяют два уровня. Первый -- это работа со сложными логическими структурами (чистая математическая логика), и второй -- работа с целыми и вещественными числами, формулами и т. д.

Идеальная ситуация, когда мы можем представить все специфические факты конкретной предметной области в виде булевых переменных, после чего натравить на них SMT-солвер, который найдёт их значения, удовлетворяющие всем ограничениям. Но на практике этого обычно недостаточно: приходится как-то интерпретировать исходные условия, и даже подбирать специальные солверы под конкретную "математику", которая хорошо помогает в решении отдельных условий. Если доказано, что они выполняются, мы их исключаем из оригинальной задачи, и продолжаем решение. Если доказано, что они не выполняются,  значит, решения нету.

Собственно, из такого подхода и родилось название **Satisfiability Module Theories** -- как уже говорилось, разрешимость логических формул с учётом лежащих в их основе теорий (теории целых и вещественных чисел, теории списков, массивов, битовых векторов и т. д.). В целом, "чистый" SMT-солвинг и обращение к конкретным предметным теориям очень тесно переплетены.

## 1.5. Что Z3 делает хорошо, а что плохо

Z3, в частности, особенно любит **конъюнктивные формулы** (CNF, связки через AND), а дополняющая их **дизъюнкция** (OR) добавляет ветвление, новые варианты.

Z3 умеет хорошо работать с такими темами:

- пропозициональная логика (непосредственно SMT-солвинг);
- линейные уравнения;
- линейные неравенства;
- полиномиальные равенства;
- битовые векторы;
- массивы;
- неинтерпретируемые функции.

В сумме они дают очень мощную и при этом на удивление гибкую базу для прикладного использования.

Есть конечно области, в которых Z3 слаб. Например, он плохо работает с **факторизацией больших чисел**, которая активно используется в криптографии. Это понятно: тут не нужна "мыслительная" логика, фактически получается тупой перебор вариантов.

Нелинейные уравнения в целом тоже трудны для решения. Z3 быстро сдаётся, если вы попытаетесь найти решение или доказательство экспоненциальной проблемы. Так же не по силам Z3 синусы, косинусы и логарифмы. Главная причина, что эти темы далеки от классической логики, для их решения нужен особый вычислительный аппарат.

In [6]:
x,y = Ints("x y")
pubkey = 3	* 7 # типа, "публичный ключ"
solve(x * y == pubkey, x > 1, y > 1) # пытаемся его взломать: успешно

[x = 3, y = 7]


In [ ]:
x,y = Ints("x y")
pubkey = 1000000993	* 1000001011 # сделаем ключ более серьёзным
# долго ждём, и не дождёмся...
solve(x * y == pubkey, x > 1, y > 1) # fail

# 2. Доказательства

## 2.1. Формальные методы

Что такое формальные методы?

"В computer science, особенно в программной и электронной инженерии, под **формальными методами** понимаются математически обоснованные техники для специфицирования, разработки/реализации и верификации программных и электронных систем".

**Спецификация** -- это точное, ясное и лаконичное изложение того, что должна делать программа. Часто спецификация имеет логический или математический "стиль".

**Реализация спецификации** -- точная, но, возможно, запутанная по причинам производительности или особенностей языка, и часто многословная. Часто она имеет программистский "стиль".

Это две стороны одной медали, разные взгляды на одно и то же. Проверкой и доказательством их эквивалентности занимается **верификация**. Но такие проверки обычно весьма сложны.

Формальная верификация реализации (программного кода) по сути представляет собой доказательство теорем. Z3 в частности позволяет делать отражение (маппинг) програмистских конструкций в подходящие математические теории.

Например, мы написали алгоритм сортировки массива, и хотим доказать, что он гарантированно будет выдавать результат, где все значения упорядочены по возрастанию или убыванию.

Во многих, если не во всех, системах верификации применяется Z3 или иные солверы, похожие на него.

## 2.2. Как Z3 доказывает теоремы

Z3 умеет доказывать теоремы. Например, мы хотим проверить, следует ли из истинности логического выражения `p and q` то, что p истинно? Очевидно, это должно быть так, так как результат and будет истинен, только если оба аргумента истинны.

Логическое следствие реализует, как уже говорилось, операция Implies. У нас есть исходная посылка And(p, q), и мы хотим вывести из неё истинность p, для чего достаточно указать в качестве выражения одну переменную p.

In [5]:
p = Bool("p")
q = Bool("q")
my_true_thm = Implies(And(p,q), p) # наша теорема, сформулированная в виде логической формулы
my_true_thm

Implies(And(p, q), p)

Доказательство выполняется вызовом метода `prove()`. Он выдаёт `proved`, если доказательство найдено.

In [7]:
prove(my_true_thm) # можно доказать?

proved


А если доказать истинность утверждения не удастся, Z3 покажет соответствующий **контрпример**, для которого оно неистинно. Например, если заменить And на Or, то вполне может получиться, что p окажется false.

In [9]:
my_false_thm = Implies(Or(p,q), p)
my_false_thm

Implies(Or(p, q), p)

In [8]:
prove(my_false_thm)

counterexample
[p = False, q = True]


### Задание

Найдите корень уравнения `x**3 + 3*x**2 + 4*x + 2 == 0` с помощью Z3. Сможете вы применить Z3 так, чтобы доказать, что найденный корень единственный, или опровергнуть это утверждение?

In [ ]:
x = Real("x")
solve(x**3 + 3*x**2 + 4*x + 2 == 0)

## 2.3. Доказательство = исчёрпывающее отсутствие контрпримеров

Итак, мы можем эффективно использовать Z3 для поиска решений для множества условий/ограничений/формул. Z3 так продуктивно этим занимается, что может полноценно определить, что никаких решений для некоторой системы не существует. Этот факт можно использовать для доказательства теорем.

Если мы сформулируем теорему `Implies(And(p,q),p)`, которая будет истинна для всех значений `p` и `q`, то сможем доказать её истинность, проведя исчерпывающий поиск контрпримера, и не найдя его. Контрпример -- это такой набор значений переменных, при которых теорема принимает значение false, или, что тоже самое, для которой отрицание `Not(th)` выдаст true.

Когда метод `solver.check` выдаёт sat, это означет, что что солвер нашёл решение, и мы можем выяснить его через доступ к solver.model. Значение unknown показывает, что "я пытался, но не смог". Возможно, решение есть, а может быть, его нет. И значение unsat показывает, что "Я гарантирую, что решения нету. Я перепробовал всё".

Вот простой пример удобной функции `prove`, где мы выполняем инверсию и интерпретируем `unsat` как доказательство.

In [ ]:
def prove2(f): # упрощенная версия функции prove() Z3
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("доказано")
    else:
        print("опровергнуто")

p = Bool("p")
q = Bool("q")
thm = Implies(And(p,q), p)
prove2(thm)

## 2.4. Доказательство пропозиционных формул

Как уже говорилось, пропозиционная формула -- это логическая формула с булевыми переменными и логическими связками между ними (and, or, not и implies). Концептуально Z3 проверяет все возможные комбинации значений true и false для всех переменных. Если однажды формула вычислится как true, Z3 вернёт `proved`.


Существуют так называемые правила де Моргана, которые утверждают, что:

-- отрицание конъюнкции есть дизъюнкция отрицаний `p & q == ~ (~p | ~q)`

-- отрицание дизъюнкции есть конъюнкция отрицаний `p | q == ~ (~p & ~q)`


In [ ]:
prove( And(p,q) == Not(Or(Not(p),Not(q))))
prove2( And(p,q) == Not(Or(Not(p),Not(q))))

Докажите второе правило самостоятельно.

## 2.5. Доказательства в арифметике

Докажите (или найдите контрпримеры) следующие теоремы для целых, рациональных, вещественных и битовых чисел:

- `x + y == y + x`

- `(x + y) + z == x + (y + z)`

- x может быть меньше, больше или равен 0;

- x в квадрате всегда положителен.


In [ ]:
# Решение

# целые
x = Int("x")
y = Int("y")
z = Int("z")

'''
# рациональные
x = Real("x")
y = Real("y")
z = Real("z")

# вещественные
x = FP("x", Float64())
y = FP("y", Float64())
z = FP("z", Float64())

# битовые
x = BitVec("x", 64)
y = BitVec("y", 64)
z = BitVec("z", 64)

'''

prove(x + y == y + x) # коммутативность
prove(((x + y) + z) == ((x + (y + z)))) # ассоциативность
prove(x + 0 == x) # существование "0"
prove(1 * x == x) # существование "1"
prove(Or(x > 0, x < 0, x == 0))
prove(x * (y + z) == x * y + x * z)
prove(x ** 2 >= 0)


## 2.6. Векторная и линейная алгебра

Используем массивы numpy для организации быстрой работы с векторами в Z3.


In [ ]:
import numpy as np

x,y = Reals("x y")
q = np.array([x,y]) # скрещиваем numpy с типами Z3

np.dot(q, q) # скалярное произведение массивов
np.array([x]) * q # скалярное произведение
q * x # скалярное произведение

np.RealVal = np.vectorize(RealVal) # маппинг типа RealVal на массив

import operator as op

# равенство векторов
def vec_eq(x,y):
    return And(np.vectorize(op.eq)(x,y).tolist())

# вектор несвязанных переменных
def NPArray(n, prefix=None, dtype=RealSort()):
    return np.array( [FreshConst(dtype, prefix=prefix) for i in range(n)] )

u = NPArray(2)
v = NPArray(2)
w = NPArray(2)

# доказываем правильность нашей векторной алгебры над типом NPArray:

prove(vec_eq(   v + u       ,  u + v            )) # коммутативность
prove(vec_eq(   v + (u + w) , (v + u) + w       )) # ассоциативность
prove(vec_eq(  (v + w) * x  , (w * x) + (v * x) )) # дистрибутивность
prove(np.dot(v, u)**2 <= np.dot(v,v) * np.dot(u,u)) # неравенство Коши — Буняковского

# дополнительно докажем свойство линейного оператора над матрицей

def Z(x):
    z = np.RealVal( np.arange(4).reshape(2,2) )
    return z @ x

def linear(Z):
    x = NPArray(2)
    y = NPArray(2)
    return vec_eq(Z(x + y), Z(x) + Z(y))

p = linear(Z)
prove(p)

## 2.7. Теорема об угле, опирающемся на диаметр окружности

Дана точка A, лежащая на окружности. Она соединена отрезками с двумя конечными точками диаметра этой окружности (тоже лежащими на окружности). Будет ли угол A прямым?

- Точка на окружности ограничена условием `x**2 + y**2 == R**2`.

- Прямой угол может быть определён как перпендикулярность, или скалярное произведение векторов, равное нулю.

In [ ]:
x, y = Reals("x y") # точка на окружности

# окружность с центром в 0,0

# если y=0, то окружность пересечёт абсциссу в точках -1 и +1

d1 = (x - 1, y) # вектор для отрезка 1
d2 = (x + 1, y) # вектор для отрезка 2

perp = d1[0]*d2[0]+d1[1]*d2[1] == 0 # условие на перпендикулярность векторов
prove(Implies(x**2 + y**2 == 1, perp))

## 2.8. Степень двойки

Данный низкоуровневый хак очень популярен среди Си-программистов. Как проверить, является ли число степенью двойки? Для этого используется следующая формула с использованием битовых операций: `x != 0 && !(x & (x - 1))`.

Мы "докажем" с помощью Z3, что эта формула будет истинна тогда и только тогда, когда x есть степень двойки. Доказательство условное потому, что мы просто сравним результаты битовых операций с правильными значениями.


In [ ]:
x      = BitVec('x', 32) # исходное число -- 32-разрядое
powers = [ 2**i for i in range(32) ] # массив степеней двойки
fast   = And(x != 0, x & (x - 1) == 0) # доказываемая формула
slow   = Or([ x == p for p in powers ]) # проверка, получился ли результат одной из степеней двойки

# доказываем, что для всех заданных значений формула истинна
prove(fast == slow) # Ok

fast   = x & (x - 1) == 0 # изменим формулу, сделаем её некорректной
prove(fast == slow) # неудачно

## 2.9. Противоположные знаки

Другой простой хак, проверяющий, будут ли у двух битовых (машинных) чисел противоположные знаки. Теперь доказательство будет формальным и полноценным.

In [ ]:
x      = BitVec('x', 32) # исходные значения
y      = BitVec('y', 32)

# Хак: (x ^ y) < 0 если x и y противоположные знаки
trick  = (x ^ y) < 0

# проверка в "лоб"
opposite = Or(And(x < 0, y >= 0),
              And(x >= 0, y < 0))

prove(trick == opposite)

## 2.10. Вавилонский метод нахождения квадратного корня

Этот простой метод подразумевает выполнение итераций по формуле
$ x_{n+1} = \frac{1}{2}(\frac{r}{x_n} + x_n)$

Мы докажем, что после 7 итераций для x в диапазоне от 0 до 10 он обеспечит точность не менее 0.1.

In [ ]:
def babylonian(x): # формула итерации
    res = 1
    for i in range(7):
        res = (x / res + res) / 2
    return res

x, y = Reals("x y")
prove(Implies(And(y**2 == x, y >= 0, 0 <= x, x <= 10), babylonian(x) - y <= 0.01))